In [1]:
from copy import deepcopy
import os

import numpy as np 
import matplotlib.pyplot as plt
import torch
import seaborn as sns

from constants import *
from utils import *
from game import *
from nn import *

In [ ]:
===== Teeko =====

TODO:
+ neural net gradient test
+ move test game to test.py


+ training loop
+ basic policy gradient
+ MCTS outline

In [2]:
path = os.getcwd()
datapath = os.path.join(path, "games")

if not os.path.exists(datapath):
    os.mkdir(datapath)

In [3]:
moves = [(3, 0), (2,1), (2,0), (1,0), (3,2), (3,1), (4,1), (2,2),
((2,0), (1,1)),
((1,0), (2,0)),
((1,1), (1,2)),
((2,2), (1,1)),
((1,2), (0,1)),
((3,1), (2,2)),
((0,1), (1,2)),
((2,2), (2,3)),
((1,2), (1,3)),
((1,1), (2,2)),]

In [4]:
game = Teeko()

for m in moves:
    if game.phase == "placing":
        action = int_from_coord(m)
        game.act(action)

    elif game.phase == "moving":
        action = int_from_move(m)
        game.act(action)

print(game.phase)
print(game.black_grid)
print(game.game_length)
print(game.game_over)

moving
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]
18
True


In [5]:
game.winner

'red'

In [13]:
temperature = 2
softmax = torch.nn.Softmax()
head = Network()

for i in range(50):
    game = Teeko()
    game_ID = f"{i}"

    generation_path = os.path.join(datapath, "gen_0")

    if not os.path.exists(generation_path):
        os.mkdir(generation_path)

    board_history = []
    action_history = []

    head.eval()

    for i in range(80):
        # get board input to NN
        board = game.represent()

        board_history.append(board)

        board = torch.tensor(np.expand_dims(board, 0), dtype=torch.float)
        prompt = game.phase

        # NN output
        value, action = head(board, prompt)

        # apply move mask to raw output, then apply temperature are softmax
        move_mask = torch.tensor(game.move_mask, dtype=torch.float)
        masked_actions = move_mask * action * temperature

        action = softmax(masked_actions)

        # cast to np
        value_ = value.detach().numpy()
        action_ = action.detach().numpy()

        # choose action on weighed distribution
        action_sum = np.cumsum(action_)
        action_int = draw_from(action_sum)

        action_history.append(action_int)

        game.act(action_int)

        if game.game_over:
            break

    board_history = np.array(board_history)  # (*, 3, 5, 5)
    action_history = np.array(action_history)  # (80,)
    winner = game.winner
    outcome = 0.5

    if winner == "black":
        outcome = 1
    elif winner == "red":
        outcome = 0

    savepath = os.path.join(generation_path, game_ID)

    np.savez(savepath, boards=board_history, actions=action_history, outcome=outcome)

In [14]:
games = os.listdir(generation_path)

for filename in games:
    filepath = os.path.join(generation_path, filename)

    gamedata = np.load(filepath)
    boards = gamedata['boards']
    actions = gamedata['actions']
    outcome = gamedata['outcome']

    print(outcome)

0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [16]:
boards.shape, actions.shape

((9, 3, 5, 5), (9,))